In [1]:
# manipulacion de archivos
import os
import json
import warnings

# manipulacion y transformacion de datos
import numpy as np
print("numpy ==",np.__version__)
import pandas as pd
print("pandas ==",pd.__version__)
import geopandas as geopd
print("geopandas ==",geopd.__version__)
import pyogrio

numpy == 1.26.0
pandas == 2.0.3
geopandas == 0.9.0


In [2]:
#from google.colab import files # solo para google collab
from google.cloud import storage

# google collab
# solo se ejecuta la primera vez: cargo el json con las credenciales del api key
#object_uploaded = files.upload()

# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [3]:
# para entorno Google Colab o maquina local:
bucket = client_storage.get_bucket('1_transform')
data_blob = bucket.blob('gmaps_metadata_filtrado.parquet')
data_gmaps_metadata = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_gmaps_metadata.info()
data_gmaps_metadata.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 441868 entries, 11 to 3024974
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              441868 non-null  object 
 1   address           439290 non-null  object 
 2   gmap_id           441868 non-null  object 
 3   description       197047 non-null  object 
 4   latitude          441868 non-null  float64
 5   longitude         441868 non-null  float64
 6   category          441868 non-null  object 
 7   avg_rating        441868 non-null  float64
 8   num_of_reviews    441868 non-null  int64  
 9   price             244781 non-null  object 
 10  hours             401547 non-null  object 
 11  MISC              436358 non-null  object 
 12  state             402667 non-null  object 
 13  relative_results  371024 non-null  object 
 14  url               441868 non-null  object 
 15  estado_categoria  441868 non-null  object 
dtypes: float64(3), int64(1)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,estado_categoria
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,cajun restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 8PM,"[0x862722fdf07aed77:0xb2269b9a8a44a51b, 0x8627...",https://www.google.com/maps/place//data=!4m2!3...,Open
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 8PM,"[0x862722fdf07aed77:0xb2269b9a8a44a51b, 0x8627...",https://www.google.com/maps/place//data=!4m2!3...,Open


In [5]:
# para entorno Google Colab o maquina local:

os.chdir('/Users/manuel/gdrive/prgrmmng/gcloud_api_key')

prefix = 'uscensus/zcta_geo/'
dl_dir = 'zcta_geo/'

bucket = client_storage.get_bucket('0_datoscrudos')
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs:
    filename = blob.name.replace('/', '_')
    blob.download_to_filename(dl_dir + filename)

geodata_zcta = geopd.read_file(
    'zcta_geo/uscensus_zcta_geo_tl_2020_us_zcta520.shp'
    ,engine="pyogrio"
    ).iloc[:,[1,5,9]]
geodata_zcta.rename(
    columns = {"GEOID20":"zcta5_geoid",
               "ALAND20":"zcta5_arealand"},
    inplace = True)
geodata_zcta.info()
geodata_zcta.head(2)

KeyboardInterrupt: 

In [7]:
data_gmaps_metadata_geo = geopd.GeoDataFrame(
    data_gmaps_metadata, geometry = geopd.points_from_xy(
        data_gmaps_metadata.longitude, data_gmaps_metadata.latitude),
    crs="EPSG:4326"
    ).to_crs(geodata_zcta.crs)

data_gmaps_metadata_geo.info()
data_gmaps_metadata_geo.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   business_id  150346 non-null  string  
 1   latitude     150346 non-null  Float64 
 2   longitude    150346 non-null  Float64 
 3   geometry     150346 non-null  geometry
dtypes: Float64(2), geometry(1), string(1)
memory usage: 6.0 MB


,business_id,latitude,longitude,geometry
0,Pns2l4eNsfO8kk83dixA6A,34.426679,-119.711197,POINT (-119.71119 34.42668)
1,mpf3x-BjTdTEA3yCZrAYPw,38.551126,-90.335695,POINT (-90.33569 38.55113)


In [8]:
data_gmaps_metadata_zcta = geopd.sjoin(
    data_gmaps_metadata_geo,
    geodata_zcta,
    how="left"
    ,op="within"
    )
data_gmaps_metadata_zcta.info()
data_gmaps_metadata_zcta.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   business_id     150346 non-null  string  
 1   latitude        150346 non-null  Float64 
 2   longitude       150346 non-null  Float64 
 3   geometry        150346 non-null  geometry
 4   index_right     144750 non-null  float64 
 5   zcta5_geoid     144750 non-null  object  
 6   zcta5_arealand  144750 non-null  float64 
dtypes: Float64(2), float64(2), geometry(1), object(1), string(1)
memory usage: 9.5+ MB


,business_id,latitude,longitude,geometry,index_right,zcta5_geoid,zcta5_arealand
0,Pns2l4eNsfO8kk83dixA6A,34.426679,-119.711197,POINT (-119.71119 34.42668),31456.0,93101,8294077.0
1,mpf3x-BjTdTEA3yCZrAYPw,38.551126,-90.335695,POINT (-90.33569 38.55113),19731.0,63123,32255679.0
2,tUFrWirKiKi_TAnsVWINQQ,32.223236,-110.880452,POINT (-110.88045 32.22323),30323.0,85711,22316058.0
3,MTSW4McQd7CbVtyjqoe9mw,39.955505,-75.155564,POINT (-75.15557 39.95550),16227.0,19107,1408994.0
4,mWMc6_wTdE0EUBKIGXDVfA,40.338183,-75.471659,POINT (-75.47166 40.33818),15901.0,18054,43881112.0


In [9]:
#### FILTRAR POR ZONA GEOGRAFICA
# filtramos las que no pertnencen al territorio EEUU
data_gmaps_metadata_zcta_usa = data_gmaps_metadata_zcta[data_gmaps_metadata_zcta["zcta5_geoid"].notnull()].copy()

# Generamos un dummy para las zonas que estan en la zona continental de EEUU
# Defino los límites geográficos de los Estados Unidos
latitud_min = 24.396308
latitud_max = 49.384358
longitud_min = -125.000000
longitud_max = -66.934570

# Filtro lugares dentro de los límites geográficos de los Estados Unidos
# Asigno el resultado a un nuevo df
data_gmaps_metadata_zcta_usa['us_continente'] = np.where((
    (data_gmaps_metadata_zcta_usa['latitude'] >= latitud_min) &
    (data_gmaps_metadata_zcta_usa['latitude'] <= latitud_max) &
    (data_gmaps_metadata_zcta_usa['longitude'] >= longitud_min) &
    (data_gmaps_metadata_zcta_usa['longitude'] <= longitud_max)),1,0)
data_gmaps_metadata_zcta_usa.info()
data_gmaps_metadata_zcta_usa.head(2)

96.28 %


In [10]:
print(round(len(data_gmaps_metadata_zcta_usa) / len(data_gmaps_metadata_zcta) * 100,2),"%")
print(round(
    len(data_gmaps_metadata_zcta_usa.loc[
        data_gmaps_metadata_zcta_usa['us_continente'] == 1
    ]) / len(data_gmaps_metadata_zcta) * 100,2),"%")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46940 entries, 0 to 46939
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   zcta5_geoid             46940 non-null  object 
 1   zcta5_name              46940 non-null  object 
 2   pop_total               46940 non-null  int64  
 3   pop_m2f_ratio           46096 non-null  float64
 4   pop_age_median          46028 non-null  float64
 5   pop_foreign_prctng      46093 non-null  float64
 6   econ_income_median      43512 non-null  float64
 7   econ_housevalue_median  43214 non-null  float64
 8   econ_houserent_median   37983 non-null  float64
 9   pop_race2_prctng        46342 non-null  float64
 10  econ_unemp_prctng       46342 non-null  float64
 11  state_name              46940 non-null  object 
 12  state_usps              46940 non-null  object 
 13  county_name             46940 non-null  object 
dtypes: float64(8), int64(1), object(5)
mem

,zcta5_geoid,zcta5_name,pop_total,pop_m2f_ratio,pop_age_median,pop_foreign_prctng,econ_income_median,econ_housevalue_median,econ_houserent_median,pop_race2_prctng,econ_unemp_prctng,state_name,state_usps,county_name
0,00601,ZCTA5 00601,17126,97.4,43.7,NaN,15292.0,78800.0,377.0,5.185099,9.776595,puerto rico,PR,adjuntas municipio
1,00601,ZCTA5 00601,17126,97.4,43.7,NaN,15292.0,78800.0,377.0,5.185099,9.776595,puerto rico,PR,utuado municipio
